# Prototipo simulador parte 2: Intercambio gaseoso

Eventualmente, las clases creadas al final del *notebook* 1, se grabarán en un módulo con extensión `.py`, de forma que puedan ser importadas al inicio de cada notebook, así:

```python
from scipy.integrate import solve_ivp
from paciente import Paciente
from ventilador import Ventilador
```  
En ese punto, su implementación sería más o menos así:

```python
paciente_sano = Paciente()
ventilador_config = Ventilador(modo='PCV')
sim = Simulador(paciente_sano, ventilador_config)
resultados_ciclo = sim.correr_ciclo()
```

Sin embargo, por ahora se copian y se pegan esas funciones aquí, ya que este método es claro y mantiene cada cuaderno autocontenido:

In [8]:
# Librerías
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Estilo
plt.style.use('seaborn-v0_8-whitegrid')

In [9]:
class Paciente:
    """Clase que define un paciente con parámetros normales;
    para agregar pacientes con patologías, crear una subclase con modificaciones
    en los parámetros específicos que haga falta"""
    def __init__(self, R1=5, C1=0.05, R2=10, C2=0.05):
        self.R1 = R1
        self.C1 = C1
        self.R2 = R2
        self.C2 = C2
        self.E1 = 1 / self.C1
        self.E2 = 1 / self.C2

In [10]:
class Ventilador:
    """Parámetros y perfiles de ventilación mecánica."""
    def __init__(self,
                 modo: str = 'PCV',
                 PEEP: float = 5.0,
                 P_driving: float = 15.0,
                 fr: float = 20.0,
                 Ti: float = 1.0,
                 Vt: float = None):
        self.modo = modo
        self.PEEP = PEEP
        self.P_driving = P_driving
        self.fr = fr
        self.Ti = Ti
        self.T_total = 60.0 / fr
        self.Vt = Vt
        if modo == 'VCV':
            assert Vt is not None, "Se requiere Vt para modo VCV"
            self.flow_insp = Vt / Ti
        else:
            self.flow_insp = None

    def presion(self, t: float) -> np.ndarray:
        """Perfil de presión en la vía aérea según el modo y el tiempo t."""
        t_arr = np.asarray(t)
        en_insp = (t_arr % self.T_total) < self.Ti
        if self.modo == 'PCV':
            P_control = self.PEEP + self.P_driving
            return np.where(en_insp, P_control, self.PEEP)
        elif self.modo == 'VCV':
            return np.full_like(t_arr, self.PEEP)
        else:
            raise ValueError(f"Modo desconocido: {modo}")

    def flujo(self, t: float) -> np.ndarray:
        """Perfil de flujo inspirado en VCV, 0 fuera de inspiración."""
        if self.modo != 'VCV':
            return np.zeros_like(np.asarray(t))
        t_arr = np.asarray(t)
        return np.where((t_arr % self.T_total) < self.Ti, self.flow_insp, 0.0)

In [11]:
class Simulador:
    """Orquesta la simulación paciente-ventilador."""
    def __init__(self,
                 paciente: Paciente,
                 ventilador: Ventilador):
        self.paciente = paciente
        self.ventilador = ventilador

    def _modelo_edo(self, t, y):
        t_arr = np.asarray(t)
        V1, V2 = y

        # Detectar inspiración vs espiración
        en_insp = (t_arr % self.ventilador.T_total) < self.ventilador.Ti

        if self.ventilador.modo == 'VCV':
            # Flujo: flujo constante en inspiración
            flow_total = np.where(en_insp,
                                  self.ventilador.flow_insp,
                                  0.0)

            # Presión de vía aérea calculada por la ecuación de los dos compartimentos
            P_aw_insp = (
                flow_total
                + (self.paciente.E1 * V1 / self.paciente.R1)
                + (self.paciente.E2 * V2 / self.paciente.R2)
            ) / ((1.0 / self.paciente.R1) + (1.0 / self.paciente.R2))

            P_aw = np.where(en_insp,
                            P_aw_insp,
                            self.ventilador.PEEP)

        else:  # Modo PCV
            P_aw = self.ventilador.presion(t_arr)

        # Se calculan las derivadas dV/dt
        dV1_dt = (P_aw - self.paciente.E1 * V1) / self.paciente.R1
        dV2_dt = (P_aw - self.paciente.E2 * V2) / self.paciente.R2

        return [dV1_dt, dV2_dt]

    def simular(self,
                num_ciclos: int=15,
                pasos_por_ciclo: int = 100):
        """Ejecuta múltiples ciclos y devuelve t, V1 y V2 concatenados."""
        t_data, V1_data, V2_data = [], [], []
        V0 = [0.0, 0.0]
        for i in range(num_ciclos):
            t0 = i * self.ventilador.T_total
            t1 = (i + 1) * self.ventilador.T_total
            if i < num_ciclos-1: #corrección para evitar discontinuidad en t=3.0
                t_eval = np.linspace(t0, t1, pasos_por_ciclo, endpoint=False)
            else:
                t_eval = np.linspace(t0, t1, pasos_por_ciclo, endpoint=True)
            sol = solve_ivp(self._modelo_edo,
                            [t0, t1],
                            V0,
                            t_eval=t_eval)
            t_data.append(sol.t)
            V1_data.append(sol.y[0])
            V2_data.append(sol.y[1])
            V0 = sol.y[:, -1]
        t = np.concatenate(t_data)
        V1 = np.concatenate(V1_data)
        V2 = np.concatenate(V2_data)
        return t, V1, V2

    def procesar_resultados(self,
                            t: np.ndarray,
                            V1: np.ndarray,
                            V2: np.ndarray) -> dict:
        """Calcula flujo, volumen total y presión resultante."""
        flujo1 = np.gradient(V1, t)
        flujo2 = np.gradient(V2, t)
        flujo_total = flujo1 + flujo2
        Vt = V1 + V2
        if self.ventilador.modo == 'PCV':
            P_aw = self.ventilador.presion(t)
        else:
            P_aw = ((flujo_total
                     + (self.paciente.E1 * V1 / self.paciente.R1)
                     + (self.paciente.E2 * V2 / self.paciente.R2))
                    / ((1 / self.paciente.R1) + (1 / self.paciente.R2)))
        return {
            't': t,
            'V1': V1,
            'V2': V2,
            'Vt': Vt,
            'flow1': flujo1,
            'flow2': flujo2,
            'flow': flujo_total,
            'P_aw': P_aw
        }

    def graficar_resultados(self,
                            resultados: dict,
                            titulo: str = 'Simulación Pulmonar'):
        """Grafica presión, flujo total y volumen total a partir del diccionario
        de resultados."""
        t = resultados['t']
        P_aw = resultados['P_aw']
        flujo = resultados['flow']
        Vt = resultados['Vt']

        fig, axs = plt.subplots(3, 1, figsize=(15, 10), sharex=True)
        fig.suptitle(titulo, fontsize=16)

        # Presión
        axs[0].plot(t, P_aw, color='red', label='Presión (P_aw)')
        axs[0].set_ylabel('Presión (cmH2O)')
        axs[0].legend()

        # Flujo
        axs[1].plot(t, flujo, color='blue', label='Flujo Total')
        axs[1].set_ylabel('Flujo (L/s)')
        axs[1].axhline(0, color='grey', linewidth=0.8)
        axs[1].legend()

        # Volumen
        axs[2].plot(t, Vt, color='green', label='Volumen (L)')
        axs[2].set_ylabel('Volumen (L)')
        axs[2].set_xlabel('Tiempo (s)')
        axs[2].legend()

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

Como se definió durante el desarrollo del Objetivo 1, para el intercambio gaseoso se decidió implementar un modelo basado en las ecuaciones del gas alveolar, siendo la entrada principal la $\dot{V}A$, esto es, la ventilación alveolar, que es una salida del módulo mecánico, permitiendo acoplar los dos módulos.

Las ecuaciones que hacen parte de este modelo son:  
$$P_A CO_2 = \frac{\dot{V} CO_2}{\dot{V}A} \cdot K$$  
$$P_A O_2 = P_I O_2 - \frac{P_A CO_2}{R} + F$$  

Se comienza la modelación, creando una función:

In [12]:
def calcular_intercambio_gas(
    resultados: dict,
    ventilador,          # instancia de Ventilador
    V_D: float,          # volumen muerto (L)
    VCO2: float,         # producción de CO2 (L/min)
    R: float,            # cociente respiratorio (adimensional)
    FiO2: float,         # fracción inspirada de O2
    Pb: float,           # presión barométrica (mmHg)
    PH2O: float = 47.0,  # presión vapor de agua a 37°C (mmHg)
    K: float = 0.863     # constante de conversión
) -> dict:
    """
    Calcula las presiones alveolares PA_CO2 y PA_O2 y las ventilaciones minuto.

    Parámetros
    ----------
    resultados : dict
        Diccionario de simular.procesar_resultados(), con claves
        't', 'Vt', ...
    ventilador : Ventilador
        Instancia con .fr (ciclos/min) y .Vt si es VCV
    V_D : float
        Volumen muerto anatómico (L)
    VCO2 : float
        Producción de CO2 (L/min)
    R : float
        Cociente respiratorio (unitario)
    FiO2 : float
        Fracción inspirada de O2 (0–1)
    Pb : float
        Presión barométrica (mmHg)

    Devuelve
    -------
    dict con
        VE_min   : ventilación minuto total (L/min)
        VA_min   : ventilación minuto alveolar (L/min)
        PACO2_mmHg
        PAO2_mmHg
    """
    # 1. Frecuencia respiratoria
    f_resp = ventilador.fr

    # 2. Volumen tidal (L)
    if ventilador.modo == 'VCV':
        VT = ventilador.Vt
    else:
        # inferirlo de "resultados['Vt']" si es PCV u otro modo
        Vt_arr = resultados['Vt']
        VT = np.max(Vt_arr) - np.min(Vt_arr)

    # 3. Ventilaciones minuto
    VE = VT * f_resp               # L/min
    VA = (VT - V_D) * f_resp       # L/min
    if VA <= 0:
        raise ValueError("Ventilación alveolar no puede ser ≤ 0")

    # 4. Presión alveolar de CO2
    PACO2 = (VCO2 * K) / VA        # mmHg

    # 5. Presión inspirada de O2 y alveolar de O2
    PIO2 = FiO2 * (Pb - PH2O)
    PAO2 = PIO2 - PACO2 / R

    return {
        "VE_min": VE,
        "VA_min": VA,
        "PACO2_mmHg": PACO2,
        "PAO2_mmHg": PAO2
    }

Poniendo el nuevo módulo a prueba:

In [13]:
# 1. Instancias
paciente = Paciente()
vent = Ventilador(modo='VCV', PEEP=5.0, P_driving=15.0,
                       fr=20.0, Ti=1.0, Vt=0.5)
sim = Simulador(paciente, vent)

# 2. Simular mecánica respiratoria
t, V1, V2 = sim.simular(num_ciclos=10, pasos_por_ciclo=200)
res_mec = sim.procesar_resultados(t, V1, V2)

# 3. Parámetros para el intercambio
params_paciente = {"V_D": 0.15}           # 150 mL dead-space
params_metabolicos = {"VCO2": 0.2,        # L/min
                       "R":    0.8}
params_ambientales = {"FiO2": 0.21,       # aire
                       "Pb":   760.0}     # mmHg al nivel del mar

# 4. Calcular intercambio
resultado_gas = calcular_intercambio_gas(
    res_mec,
    ventilador=vent,
    V_D=params_paciente["V_D"],
    VCO2=params_metabolicos["VCO2"],
    R=params_metabolicos["R"],
    FiO2=params_ambientales["FiO2"],
    Pb=params_ambientales["Pb"]
)

print("Resultados intercambio gaseoso:")
for k, v in resultado_gas.items():
    print(f"  {k} = {v:.2f}")


Resultados intercambio gaseoso:
  VE_min = 10.00
  VA_min = 7.00
  PACO2_mmHg = 0.02
  PAO2_mmHg = 149.70


Finalmente, para cumplir con el principio de la programación pragmática DRY, se encapsula este nuevo desarrollo dentro de un módulo, con la clase `IntercambioGases`, que podrá ser llamada por los siguientes módulos. De nuevo, con esta estructura, cada módulo es autónomo y fácil de importar, modificar o poner a prueba (*testear*).

In [14]:
class IntercambioGases:
    """
    Módulo de intercambio gaseoso alveolar.
    Calcula presiones de CO2 y O2 alveolares a partir de los resultados
    de la simulación de mecánica respiratoria.
    """
    def __init__(
        self,
        ventilador,
        V_D: float,
        VCO2: float,
        R: float,
        FiO2: float,
        Pb: float,
        PH2O: float = 47.0,
        K: float = 0.863
    ):
        self.ventilador = ventilador
        self.V_D = V_D      # volumen muerto anatómico (L)
        self.VCO2 = VCO2    # producción de CO2 (L/min)
        self.R = R          # cociente respiratorio (adimensional)
        self.FiO2 = FiO2    # fracción inspirada de O2 (0-1)
        self.Pb = Pb        # presión barométrica (mmHg)
        self.PH2O = PH2O    # presión vapor de agua a 37°C (mmHg)
        self.K = K          # constante de conversión de unidades

    def calcular(self, resultados: dict) -> dict:
        """
        Ejecuta el cálculo de intercambio gaseoso.

        Parámetros
        ----------
        resultados : dict
            Diccionario de salida de Simulador.procesar_resultados(),
            con claves 't' (tiempo) y 'Vt' (volumen total alveolar, L).

        Devuelve
        -------
        dict con:
            VE_min: ventilación minuto total (L/min)
            VA_min: ventilación minuto alveolar (L/min)
            PACO2_mmHg: presión alveolar de CO2 (mmHg)
            PAO2_mmHg: presión alveolar de O2 (mmHg)
        """
        # 1. Frecuencia respiratoria (ciclos/min)
        f = self.ventilador.fr

        # 2. Volumen tidal (L)
        if self.ventilador.modo == 'VCV':
            VT = self.ventilador.Vt
        else:
            arr = resultados['Vt']
            VT = np.max(arr) - np.min(arr)

        # 3. Ventilaciones minuto
        VE = VT * f
        VA = (VT - self.V_D) * f
        if VA <= 0:
            raise ValueError("Ventilación alveolar ≤ 0, revisa V_D o simulación.")

        # 4. Presión alveolar de CO2 (mmHg)
        PACO2 = (self.VCO2 * self.K) / VA

        # 5. Presión alveolar de O2 (mmHg)
        PIO2 = self.FiO2 * (self.Pb - self.PH2O)
        PAO2 = PIO2 - (PACO2 / self.R)

        return {
            'VE_min': VE,
            'VA_min': VA,
            'PACO2_mmHg': PACO2,
            'PAO2_mmHg': PAO2
        }


Un ejemplo de implementación de este nuevo módulo:

In [15]:
# 1. Configuración y simulación de mecánica
paciente = Paciente()
vent = Ventilador(modo='VCV', PEEP=5.0, P_driving=15.0, fr=20., Ti=1.0, Vt=0.5)
sim = Simulador(paciente, vent)
t, V1, V2 = sim.simular(num_ciclos=15, pasos_por_ciclo=100)
res_mec = sim.procesar_resultados(t, V1, V2)

# 2. Intercambio gaseoso
ig = IntercambioGases(
    ventilador=vent,
    V_D=0.15,           # Volumen muerto (L)
    VCO2=0.2,           # L/min
    R=0.8,
    FiO2=0.21,
    Pb=760.0            # mmHg
)
res_gas = ig.calcular(res_mec)

print(res_gas)

{'VE_min': 10.0, 'VA_min': 7.0, 'PACO2_mmHg': 0.024657142857142857, 'PAO2_mmHg': 149.69917857142855}
